Importing relevant modules and raw data.

In [1]:
from db_utils import QueryAll
import pandas as pd

/Users/liumukun/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
columns = ["id", "url", "location", "brand", "model", "screenSize", "storage", "postage", "shippingType", 
          "endTime", "returnsAccepted", "returnsDays", "condition", "sellerFeedbackScore", "sellerPositivePercent", 
          "sellerName", "price", "bids", "ram", "resolution", "sellerPositive", "sellerNegative", "quantity", 
          "sellerItemsSold", "sellerRecordedFeedback"]
data = pd.DataFrame(QueryAll())
data.columns = columns

The ```isPolice``` column identifies whether a listing is sold by a police auctioneer.

In [3]:
data["isPolice"] = (data.sellerName == "leicester_police_property_disposa...") | (data.sellerName == "sussexpolice-auctions")
data.isPolice = data.isPolice.astype(int)

The ```ram``` column specifies the memory of the device sold, in gigabytes. Any listings with recorded RAM higher than 10 GB are treated as missing values, as those sellers mistakenly thought memory meant storage.

In [4]:
data.ram = data.ram.astype(float)
data.loc[data.ram > 10, "ram"] = -1

The ```storage``` column specifies the storage of the device sold, in gigabytes. Here it is casted to integers.

In [5]:
data.storage = data.storage.astype(int)

The ```sellerLifePercent``` column stores the percentage of positive feedback received by the seller of a listing throughout their time on eBay. The ```sellerYearPercent``` stores this percentage restricted to listings sold within the past year.

In [6]:
data.sellerPositive = data.sellerPositive.astype(str).str.replace(",","").astype(int)
data.sellerNegative = data.sellerNegative.astype(str).str.replace(",","").astype(int)

In [7]:
data["sellerLifePercent"] = data.sellerPositivePercent.str.replace("%","").astype(float)
data["sellerYearPercent"] = data.sellerPositive/(data.sellerPositive+data.sellerNegative).astype(float)*100
data.sellerYearPercent = data.sellerYearPercent.round()
data.loc[data.sellerYearPercent.isna(), "sellerYearPercent"] = 0

The ```brand``` and ```model``` columns contain specifications of the device, and here all whitespaces are removed and text made lower case.

In [8]:
data.brand = data.brand.str.lower().str.strip()
data.model = data.model.str.lower().str.strip()

The brands of listings are converted to dummy variables for major brands present.

In [9]:
data["apple"] = (data.brand == "apple").astype(int)
data["samsung"] = (data.brand == "samsung").astype(int)
data["amazon"] = (data.brand == "amazon").astype(int)
data["others"] = ((data.brand != "apple") & (data.brand != "samsung") & (data.brand != "amazon")).astype(int)

The ```sellerItemsSold``` column stores the total number of listings sold by the seller.

In [10]:
def repl(num):
    n = num.group(0)
    if "K" in n:
        if "." in n:
            return n.replace(".","").replace("K","") + "00"
        else:
            return n.replace("K","") + "000"
    elif "M" in n:
        if "." in n:
            return n.replace(".","").replace("M","") + "00000"
        else:
            return n.replace("M","") + "000000"
    
data.sellerItemsSold = data.sellerItemsSold.astype(str).str.replace("[0-9,.]+(M|K)", repl, regex=True).astype(int)

The ```postage``` column stores the cost of shipping of the listing. Free postage is recorded as 0.

In [11]:
def transform_p(po):
    p = str(po)
    if p == "Free" or p == "0":
        return 0
    elif p.startswith("£"):
        return p[1:]
    elif p.startswith("EUR"):
        return float(p[4:])*0.88

data.postage = data.postage.apply(transform_p)

The ```isUsed``` column specifies if the device sold is brand new or used. The ```isBroken``` column specifies if it is partly not working. 

In [12]:
data["isUsed"] = (data.condition == "Used") | (data.condition == "For parts or not working") \
    | (data.condition == "Good - Refurbished")
data["isBroken"] = (data.condition == "For parts or not working")
data.isUsed = data.isUsed.astype(int)
data.isBroken = data.isBroken.astype(int)

The ```deliverySpeed``` column specifies the speed of the postage service used by the seller, labeled from 0 to 2, where 0 is the slowest (economy) and 2 is the fastest (express or courier).

In [13]:
def transform_d(de):
    if de == "Economy Delivery" or de == "An Post International" or de =="Free collection in person":
        return 0
    elif de == "Standard Delivery":
        return 1
    elif de == "Express Delivery" or de == "Courier":
        return 2

data["deliverySpeed"] = data.shippingType.apply(transform_d)

The ```startingPrice``` column stores the minimum price set by the seller. This includes the postage.

In [14]:
def transform_s(row):
    b = eval(row.bids)
    _b = b[-1].replace(",","")
    if _b.startswith("£"):
        return float(b[-1][1:])+float(row.postage)
    elif _b.startswith("EUR"):
        return float(b[-1][4:])*0.88+float(row.postage)
    
data["startingPrice"] = data.apply(transform_s, axis=1)

The ```bids``` column contains arrays of highest bids made by distinct bidders, in descending order. It contains the winning price but not the minimum price, all inclusive of the postage.

In [15]:
def transform_b(row):
    b = eval(row.bids)[:-1]
    _bids = []
    for bid in b:
        _bid = bid.replace(",","")
        if bid.startswith("£"):
            _bids.append(_bid[1:])
        elif bid.startswith("EUR"):
            _bids.append(float(_bid[4:])*0.88)
        
    bids_with_postage = [float(bid) + float(row.postage) for bid in _bids]
    return bids_with_postage

data.bids = data.apply(transform_b, axis=1)

The ```isSold``` column indicates whether a listing received at least one bid.

In [16]:
data["isSold"] = data.bids.astype(bool).astype(int)

The ```price``` column records the transaction price of the listing. It is not the highest bid but an increment of the second highest, as per eBay rules. This includes the postage.

In [17]:
def transform_pr(row):
    p = str(row.price)
    _p = p.replace(",","")
    if p.startswith("£"):
        return float(_p[1:])+float(row.postage)
    elif p.startswith("EUR"):
        return float(_p[4:])*0.88+float(row.postage)

data.price = data.apply(transform_pr, axis=1)

Here, all price related columns are adjusted for the quantity of devices sold within each listing.

In [18]:
def transform_q(row):
    row.bids = [round(float(bid)/row.quantity, 2) for bid in row.bids]
    row.startingPrice = round(float(row.startingPrice)/row.quantity, 2)
    row.price = round(float(row.price)/row.quantity, 2)
    return row
    
data = data.apply(transform_q, axis=1)

The ```bidCount``` column stores the number of bids a auction received.

In [19]:
data["bidCount"] = data.bids.apply(lambda a: len(a))

In [20]:
data.endTime

0       08 Dec, 2022 20:12:01 GMT
1       12 Dec, 2022 22:37:46 GMT
2       17 Dec, 2022 11:01:15 GMT
3       13 Dec, 2022 13:57:13 GMT
4       14 Dec, 2022 20:50:56 GMT
                  ...            
1059    10 Feb, 2023 20:48:54 GMT
1060    08 Jan, 2023 22:40:01 GMT
1061    11 Jan, 2023 18:14:29 GMT
1062    28 Feb, 2023 22:25:37 GMT
1063    08 Dec, 2022 18:02:58 GMT
Name: endTime, Length: 1064, dtype: object

The ```isNight``` column stores whether an auction ended during night time, defined as being after 5PM and before 2AM.

In [21]:
import dateutil

tzmapping = {"GMT": dateutil.tz.gettz("Europe/London"),
             "BST": dateutil.tz.gettz("Europe/London")}

data.endTime = data.endTime.apply(dateutil.parser.parse, tzinfos=tzmapping).dt.tz_convert("UTC")

data.endTime = pd.to_datetime(data.endTime)
data["isNight"] = ((data.endTime.dt.hour > 16) | (data.endTime.dt.hour < 2)).astype(int)

In [22]:
data.sellerRecordedFeedback = data.sellerRecordedFeedback.astype(str).str.replace(",","").astype(int)

In [23]:
import statsmodels.api as sm

X = data[data["sellerFeedbackScore"] >= 0]["sellerRecordedFeedback"]
y = data[data["sellerFeedbackScore"] >= 0]["sellerFeedbackScore"]

exog = sm.add_constant(X)

result = sm.OLS(y,exog).fit()

data.loc[data[data["sellerFeedbackScore"] == -1].index,"sellerFeedbackScore"] = \
data[data["sellerFeedbackScore"] == -1].sellerRecordedFeedback.apply(lambda n: result.predict([1,n])[0])

The data is exported as a ```.csv``` file for further analysis.

In [24]:
keep = ["id", "apple", "samsung", "amazon", "others", "model", "screenSize", "storage", "isNight", "returnsAccepted", "returnsDays", 
           "sellerFeedbackScore", "bids", "price", "ram", "resolution", "sellerItemsSold", "isPolice", "sellerLifePercent",
           "sellerYearPercent", "isUsed", "isBroken", "deliverySpeed", "startingPrice", "isSold", "bidCount"]
data = data[keep]

In [25]:
data.to_csv("../data/listings.csv", index=False)